<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

In [2]:
import sys
import os
import logging
import papermill as pm
import scrapbook as sb
from tempfile import TemporaryDirectory
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

from recommenders.datasets.amazon_reviews import download_and_extract, data_preprocessing
from recommenders.datasets.download_utils import maybe_download

from recommenders.models.deeprec.models.sequential.sli_rec import SLI_RECModel as SeqModel
from recommenders.models.deeprec.io.sequential_iterator import SequentialIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]
Tensorflow version: 1.15.0


# 1. Input Data Format
The input data contains 8 columns, i.e.,   `<label> <user_id> <item_id> <category_id> <timestamp> <history_item_ids> <history_cateory_ids> <hitory_timestamp>`  columns are seperated by `"\t"`.  item_id and category_id denote the target item and category, which means that for this instance, we want to guess whether user user_id will interact with item_id at timestamp. `<history_*>` columns record the user behavior list up to `<timestamp>`, elements are separated by commas.  `<label>` is a binary value with 1 for positive instances and 0 for negative instances.  One example for an instance is: 

`1	523120	14414	3	1612282953577	14330,14135,5877	3,0,8	1611878652251,1611935700202,1612166343656`

In data preprocessing stage, we have a script to generate some ID mapping dictionaries, so user_id, item_id and category_id will be mapped into interager index starting from 1. And you need to tell the input iterator where is the ID mapping files are. (For example, in the next section, we have some mapping files like user_vocab, item_vocab, and cate_vocab).  The data preprocessing script is at [recommenders/dataset/amazon_reviews.py](../nhat_hoang/recommenders/datasets/amazon_reviews.py). Note that ID vocabulary only creates from the train_file, so the new IDs in valid_file or test_file will be regarded as unknown IDs and assigned with a defualt 0 index.

We use Softmax to the loss function. In training and evalution stage, we group 1 positive instance with num_ngs negative instances. Pair-wise ranking can be regarded as a special case of Softmax ranking, where num_ngs is set to 1. 

More specifically,  for training and evalation, you need to organize the data file such that each one positive instance is followd by num_ngs negative instances. Our program will take 1+num_ngs lines as a unit for Softmax calculation. num_ngs is a parameter you need to pass to the `prepare_hparams`, `fit` and `run_eval` function. `train_num_ngs` in `prepare_hparams` denotes the number of negative instances for training, where a recommended number is 4. `valid_num_ngs` and `num_ngs` in `fit` and `run_eval` denote the number in evalution. In evaluation, the model calculates metrics among the 1+num_ngs instances. For the `predict` function, since we only need to calcuate a socre for each individual instance, there is no need for num_ngs setting.  More details and examples will be provided in the following sections.

In [3]:
yaml_file = '../nhat_hoang/recommenders/models/deeprec/config/sli_rec.yaml'

In [4]:
EPOCHS = 10
BATCH_SIZE = 400
RANDOM_SEED = SEED  # Set None for non-deterministic result

data_path = os.path.join("..", "nhat_hoang", "test_slirec", "whole_dataset2")

In [5]:
# for test
train_file = os.path.join(data_path, r'train_data')
valid_file = os.path.join(data_path, r'valid_data')
test_file = os.path.join(data_path, r'test_data')
user_vocab = os.path.join(data_path, r'user_vocab.pkl')
item_vocab = os.path.join(data_path, r'item_vocab.pkl')
cate_vocab = os.path.join(data_path, r'category_vocab.pkl')
output_file = os.path.join(data_path, r'output.txt')

train_num_ngs = 4 # number of negative instances with a positive instance for training
valid_num_ngs = 4 # number of negative instances with a positive instance for validation
test_num_ngs = 9 # number of negative instances with a positive instance for testing
sample_rate = 1 # sample a small item set for training and testing here for fast example

if not os.path.exists(train_file):
    data_preprocessing(train_file, valid_file, test_file, user_vocab, item_vocab, cate_vocab,
                       sample_rate=sample_rate, valid_num_ngs=valid_num_ngs, test_num_ngs=test_num_ngs)

### 1.1 Prepare hyper-parameters

`prepare_hparams()` will create a full set of hyper-parameters for model training, such as learning rate, feature number, and dropout ratio. We can put those parameters in a yaml file (a complete list of parameters can be found under our config folder) , or pass parameters as the function's parameters (which will overwrite yaml settings).

Parameters hints:
need_sample controls whether to perform dynamic negative sampling in mini-batch. train_num_ngs indicates how many negative instances followed by one positive instances.
Examples:
- `need_sample=True` and `train_num_ngs=4`: There are only positive instances in your training file. Our model will dynamically sample 4 negative instances for each positive instances in mini-batch. Note that if `need_sample is` set to True, `train_num_ngs` should be greater than zero.
- `need_sample=False` and `train_num_ngs=4`: In your training file, each one positive line is followed by 4 negative lines. Note that if `need_sample` is set to False, you must provide a traiing file with negative instances, and `train_num_ngs` should match the number of negative number in training file.

In [7]:
hparams = prepare_hparams(yaml_file, 
                          embed_l2=0., 
                          layer_l2=0., 
                          learning_rate=0.001,  # set to 0.01 if batch normalization is disable
                          epochs=EPOCHS,
                          batch_size=BATCH_SIZE,
                          show_step=20,
                          MODEL_DIR=os.path.join(data_path, "model/"),
                          SUMMARIES_DIR=os.path.join(data_path, "summary/"),
                          user_vocab=user_vocab,
                          item_vocab=item_vocab,
                          cate_vocab=cate_vocab,
                          need_sample=True,
                          train_num_ngs=train_num_ngs, # provides the number of negative instances for each positive instance for loss computation.
)

### 1.2 Create data loader
Designate a data iterator for the model. All our sequential models use SequentialIterator. data format is introduced aboved.


Validation and testing data are files after negative sampling offline with the number of <num_ngs> and <test_num_ngs>.

In [8]:
input_creator = SequentialIterator

# 2. Create Model

In [9]:
model = SeqModel(hparams, input_creator, seed=RANDOM_SEED)

# model.load_model(os.path.join(hparams.MODEL_DIR, "best_model"))

Performance before training

In [10]:
model.run_eval(test_file, num_ngs=test_num_ngs)

{'auc': 0.5268,
 'logloss': 0.6931,
 'mean_mrr': 0.283,
 'group_auc': 0.526,
 'ndcg@1': 0.0829,
 'ndcg@3': 0.2088,
 'ndcg@5': 0.2865,
 'hit@1': 0.0829,
 'hit@3': 0.3059,
 'hit@5': 0.4956}

AUC around 0.5 is a state of random guess. We can see that before training, the model behaves like random guessing.

### Train model

In [11]:
with Timer() as train_time:
    model = model.fit(train_file, valid_file, valid_num_ngs=valid_num_ngs) 

# valid_num_ngs is the number of negative lines after each positive line in your valid_file 
# we will evaluate the performance of model on valid_file every epoch
print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

step 20 , total_loss: 1.3830, data_loss: 1.3830
step 40 , total_loss: 1.1761, data_loss: 1.1761
step 60 , total_loss: 1.1281, data_loss: 1.1281
step 80 , total_loss: 1.0655, data_loss: 1.0655
step 100 , total_loss: 1.0706, data_loss: 1.0706
step 120 , total_loss: 1.0018, data_loss: 1.0018
step 140 , total_loss: 1.0814, data_loss: 1.0814
step 160 , total_loss: 1.0410, data_loss: 1.0410
step 180 , total_loss: 1.0311, data_loss: 1.0311
step 200 , total_loss: 0.9807, data_loss: 0.9807
step 220 , total_loss: 1.0163, data_loss: 1.0163
step 240 , total_loss: 1.1000, data_loss: 1.1000
step 260 , total_loss: 1.0024, data_loss: 1.0024
step 280 , total_loss: 1.0444, data_loss: 1.0444
step 300 , total_loss: 0.9967, data_loss: 0.9967
step 320 , total_loss: 1.0112, data_loss: 1.0112
step 340 , total_loss: 0.9862, data_loss: 0.9862
step 360 , total_loss: 1.0074, data_loss: 1.0074
step 380 , total_loss: 0.9702, data_loss: 0.9702
step 400 , total_loss: 1.0179, data_loss: 1.0179
step 420 , total_loss: 0

step 3320 , total_loss: 0.9628, data_loss: 0.9628
step 3340 , total_loss: 0.8218, data_loss: 0.8218
step 3360 , total_loss: 0.8483, data_loss: 0.8483
step 3380 , total_loss: 0.8602, data_loss: 0.8602
step 3400 , total_loss: 0.9133, data_loss: 0.9133
step 3420 , total_loss: 0.8697, data_loss: 0.8697
step 3440 , total_loss: 0.8781, data_loss: 0.8781
step 3460 , total_loss: 0.9064, data_loss: 0.9064
step 3480 , total_loss: 0.8939, data_loss: 0.8939
step 3500 , total_loss: 0.8418, data_loss: 0.8418
step 3520 , total_loss: 0.9043, data_loss: 0.9043
step 3540 , total_loss: 0.8915, data_loss: 0.8915
step 3560 , total_loss: 0.9823, data_loss: 0.9823
step 3580 , total_loss: 0.8851, data_loss: 0.8851
step 3600 , total_loss: 0.9728, data_loss: 0.9728
step 3620 , total_loss: 0.9125, data_loss: 0.9125
step 3640 , total_loss: 0.9006, data_loss: 0.9006
step 3660 , total_loss: 0.8952, data_loss: 0.8952
step 3680 , total_loss: 0.8508, data_loss: 0.8508
step 3700 , total_loss: 0.8621, data_loss: 0.8621


step 6600 , total_loss: 0.9079, data_loss: 0.9079
step 6620 , total_loss: 0.8478, data_loss: 0.8478
step 6640 , total_loss: 0.8078, data_loss: 0.8078
step 6660 , total_loss: 0.7753, data_loss: 0.7753
step 6680 , total_loss: 0.8543, data_loss: 0.8543
step 6700 , total_loss: 0.8305, data_loss: 0.8305
step 6720 , total_loss: 0.8191, data_loss: 0.8191
step 6740 , total_loss: 0.8543, data_loss: 0.8543
step 6760 , total_loss: 0.7993, data_loss: 0.7993
step 6780 , total_loss: 0.7937, data_loss: 0.7937
step 6800 , total_loss: 0.7861, data_loss: 0.7861
step 6820 , total_loss: 0.8581, data_loss: 0.8581
step 6840 , total_loss: 0.8703, data_loss: 0.8703
step 6860 , total_loss: 0.8369, data_loss: 0.8369
step 6880 , total_loss: 0.8527, data_loss: 0.8527
step 6900 , total_loss: 0.8610, data_loss: 0.8610
step 6920 , total_loss: 0.8407, data_loss: 0.8407
step 6940 , total_loss: 0.8254, data_loss: 0.8254
step 6960 , total_loss: 0.8752, data_loss: 0.8752
step 6980 , total_loss: 0.7968, data_loss: 0.7968


step 2000 , total_loss: 0.7822, data_loss: 0.7822
step 2020 , total_loss: 0.7548, data_loss: 0.7548
step 2040 , total_loss: 0.8345, data_loss: 0.8345
step 2060 , total_loss: 0.6943, data_loss: 0.6943
step 2080 , total_loss: 0.7346, data_loss: 0.7346
step 2100 , total_loss: 0.8672, data_loss: 0.8672
step 2120 , total_loss: 0.7716, data_loss: 0.7716
step 2140 , total_loss: 0.7223, data_loss: 0.7223
step 2160 , total_loss: 0.7588, data_loss: 0.7588
step 2180 , total_loss: 0.7200, data_loss: 0.7200
step 2200 , total_loss: 0.8247, data_loss: 0.8247
step 2220 , total_loss: 0.7805, data_loss: 0.7805
step 2240 , total_loss: 0.8345, data_loss: 0.8345
step 2260 , total_loss: 0.8818, data_loss: 0.8818
step 2280 , total_loss: 0.7672, data_loss: 0.7672
step 2300 , total_loss: 0.7762, data_loss: 0.7762
step 2320 , total_loss: 0.7559, data_loss: 0.7559
step 2340 , total_loss: 0.7824, data_loss: 0.7824
step 2360 , total_loss: 0.8188, data_loss: 0.8188
step 2380 , total_loss: 0.7511, data_loss: 0.7511


step 5280 , total_loss: 0.6931, data_loss: 0.6931
step 5300 , total_loss: 0.7506, data_loss: 0.7506
step 5320 , total_loss: 0.7681, data_loss: 0.7681
step 5340 , total_loss: 0.7621, data_loss: 0.7621
step 5360 , total_loss: 0.8249, data_loss: 0.8249
step 5380 , total_loss: 0.8009, data_loss: 0.8009
step 5400 , total_loss: 0.7016, data_loss: 0.7016
step 5420 , total_loss: 0.8263, data_loss: 0.8263
step 5440 , total_loss: 0.7796, data_loss: 0.7796
step 5460 , total_loss: 0.7309, data_loss: 0.7309
step 5480 , total_loss: 0.7897, data_loss: 0.7897
step 5500 , total_loss: 0.8172, data_loss: 0.8172
step 5520 , total_loss: 0.7351, data_loss: 0.7351
step 5540 , total_loss: 0.7691, data_loss: 0.7691
step 5560 , total_loss: 0.7952, data_loss: 0.7952
step 5580 , total_loss: 0.7376, data_loss: 0.7376
step 5600 , total_loss: 0.7751, data_loss: 0.7751
step 5620 , total_loss: 0.7854, data_loss: 0.7854
step 5640 , total_loss: 0.7377, data_loss: 0.7377
step 5660 , total_loss: 0.7804, data_loss: 0.7804


...

step 2660 , total_loss: 0.7204, data_loss: 0.7204
step 2680 , total_loss: 0.7621, data_loss: 0.7621
step 2700 , total_loss: 0.7473, data_loss: 0.7473
step 2720 , total_loss: 0.6815, data_loss: 0.6815
step 2740 , total_loss: 0.7304, data_loss: 0.7304
step 2760 , total_loss: 0.7403, data_loss: 0.7403
step 2780 , total_loss: 0.6658, data_loss: 0.6658
step 2800 , total_loss: 0.6986, data_loss: 0.6986
step 2820 , total_loss: 0.6805, data_loss: 0.6805
step 2840 , total_loss: 0.7905, data_loss: 0.7905
step 2860 , total_loss: 0.7200, data_loss: 0.7200
step 2880 , total_loss: 0.7194, data_loss: 0.7194
step 2900 , total_loss: 0.7234, data_loss: 0.7234
step 2920 , total_loss: 0.7218, data_loss: 0.7218
step 2940 , total_loss: 0.7273, data_loss: 0.7273
step 2960 , total_loss: 0.6762, data_loss: 0.6762
step 2980 , total_loss: 0.7421, data_loss: 0.7421
step 3000 , total_loss: 0.7678, data_loss: 0.7678
step 3020 , total_loss: 0.7559, data_loss: 0.7559
step 3040 , total_loss: 0.6873, data_loss: 0.6873


step 5940 , total_loss: 0.7256, data_loss: 0.7256
step 5960 , total_loss: 0.7189, data_loss: 0.7189
step 5980 , total_loss: 0.7114, data_loss: 0.7114
step 6000 , total_loss: 0.7430, data_loss: 0.7430
step 6020 , total_loss: 0.7731, data_loss: 0.7731
step 6040 , total_loss: 0.8101, data_loss: 0.8101
step 6060 , total_loss: 0.7935, data_loss: 0.7935
step 6080 , total_loss: 0.6630, data_loss: 0.6630
step 6100 , total_loss: 0.6714, data_loss: 0.6714
step 6120 , total_loss: 0.6444, data_loss: 0.6444
step 6140 , total_loss: 0.7444, data_loss: 0.7444
step 6160 , total_loss: 0.7666, data_loss: 0.7666
step 6180 , total_loss: 0.7545, data_loss: 0.7545
step 6200 , total_loss: 0.6957, data_loss: 0.6957
step 6220 , total_loss: 0.7217, data_loss: 0.7217
step 6240 , total_loss: 0.6943, data_loss: 0.6943
step 6260 , total_loss: 0.6813, data_loss: 0.6813
step 6280 , total_loss: 0.7276, data_loss: 0.7276
step 6300 , total_loss: 0.7135, data_loss: 0.7135
step 6320 , total_loss: 0.7767, data_loss: 0.7767


step 1340 , total_loss: 0.6918, data_loss: 0.6918
step 1360 , total_loss: 0.6968, data_loss: 0.6968
step 1380 , total_loss: 0.7668, data_loss: 0.7668
step 1400 , total_loss: 0.6626, data_loss: 0.6626
step 1420 , total_loss: 0.6775, data_loss: 0.6775
step 1440 , total_loss: 0.6931, data_loss: 0.6931
step 1460 , total_loss: 0.7642, data_loss: 0.7642
step 1480 , total_loss: 0.7178, data_loss: 0.7178
step 1500 , total_loss: 0.7585, data_loss: 0.7585
step 1520 , total_loss: 0.7060, data_loss: 0.7060
step 1540 , total_loss: 0.7125, data_loss: 0.7125
step 1560 , total_loss: 0.7017, data_loss: 0.7017
step 1580 , total_loss: 0.7433, data_loss: 0.7433
step 1600 , total_loss: 0.7332, data_loss: 0.7332
step 1620 , total_loss: 0.7079, data_loss: 0.7079
step 1640 , total_loss: 0.7602, data_loss: 0.7602
step 1660 , total_loss: 0.7502, data_loss: 0.7502
step 1680 , total_loss: 0.6621, data_loss: 0.6621
step 1700 , total_loss: 0.7462, data_loss: 0.7462
step 1720 , total_loss: 0.7111, data_loss: 0.7111


step 4620 , total_loss: 0.7641, data_loss: 0.7641
step 4640 , total_loss: 0.7353, data_loss: 0.7353
step 4660 , total_loss: 0.6632, data_loss: 0.6632
step 4680 , total_loss: 0.7638, data_loss: 0.7638
step 4700 , total_loss: 0.6947, data_loss: 0.6947
step 4720 , total_loss: 0.6969, data_loss: 0.6969
step 4740 , total_loss: 0.6935, data_loss: 0.6935
step 4760 , total_loss: 0.7950, data_loss: 0.7950
step 4780 , total_loss: 0.7760, data_loss: 0.7760
step 4800 , total_loss: 0.7151, data_loss: 0.7151
step 4820 , total_loss: 0.7991, data_loss: 0.7991
step 4840 , total_loss: 0.7564, data_loss: 0.7564
step 4860 , total_loss: 0.7063, data_loss: 0.7063
step 4880 , total_loss: 0.7511, data_loss: 0.7511
step 4900 , total_loss: 0.7720, data_loss: 0.7720
step 4920 , total_loss: 0.7264, data_loss: 0.7264
step 4940 , total_loss: 0.7086, data_loss: 0.7086
step 4960 , total_loss: 0.7330, data_loss: 0.7330
step 4980 , total_loss: 0.7477, data_loss: 0.7477
step 5000 , total_loss: 0.6629, data_loss: 0.6629


[(1, {'auc': 0.8593, 'logloss': 0.5418, 'mean_mrr': 0.7493, 'group_auc': 0.8489, 'ndcg@1': 0.559, 'ndcg@3': 0.7915, 'ndcg@5': 0.8133, 'hit@1': 0.559, 'hit@3': 0.9479, 'hit@5': 1.0}), (2, {'auc': 0.8684, 'logloss': 0.6311, 'mean_mrr': 0.7586, 'group_auc': 0.8566, 'ndcg@1': 0.5733, 'ndcg@3': 0.8005, 'ndcg@5': 0.8202, 'hit@1': 0.5733, 'hit@3': 0.9529, 'hit@5': 1.0}), (3, {'auc': 0.8708, 'logloss': 0.7495, 'mean_mrr': 0.7611, 'group_auc': 0.8586, 'ndcg@1': 0.5772, 'ndcg@3': 0.8028, 'ndcg@5': 0.8221, 'hit@1': 0.5772, 'hit@3': 0.9539, 'hit@5': 1.0}), (4, {'auc': 0.8722, 'logloss': 0.868, 'mean_mrr': 0.7634, 'group_auc': 0.8604, 'ndcg@1': 0.5809, 'ndcg@3': 0.8049, 'ndcg@5': 0.8239, 'hit@1': 0.5809, 'hit@3': 0.9547, 'hit@5': 1.0}), (5, {'auc': 0.8734, 'logloss': 1.0367, 'mean_mrr': 0.7645, 'group_auc': 0.8609, 'ndcg@1': 0.5831, 'ndcg@3': 0.8056, 'ndcg@5': 0.8246, 'hit@1': 0.5831, 'hit@3': 0.9545, 'hit@5': 1.0}), (6, {'auc': 0.8739, 'logloss': 1.1299, 'mean_mrr': 0.7657, 'group_auc': 0.8617, 'n

### Evaluate the performance after training

In [12]:
model.run_eval(test_file, num_ngs=test_num_ngs)

{'auc': 0.8731,
 'logloss': 2.2132,
 'mean_mrr': 0.6169,
 'group_auc': 0.8618,
 'ndcg@1': 0.3867,
 'ndcg@3': 0.6416,
 'ndcg@5': 0.6917,
 'hit@1': 0.3867,
 'hit@3': 0.8189,
 'hit@5': 0.9398}